In [2]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from tensorflow.python.client import device_lib
device_lib.list_local_devices() 

Using TensorFlow backend.


[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17854016837411432204, name: "/gpu:0"
 device_type: "GPU"
 memory_limit: 2545287168
 locality {
   bus_id: 1
 }
 incarnation: 13325176528434444270
 physical_device_desc: "device: 0, name: GeForce GTX 780, pci bus id: 0000:02:00.0"]

In [3]:
def get_shuffled_splitted_data(path):
    df = pd.read_csv(path)

    # Shuffle and split data
    X_train, X_test, X_val = np.split(df.sample(frac=1), [int(.7*len(df)), int(.9*len(df))])
    
    # Pop labels and transform them to vectors
    y_train, y_test, y_val = X_train.pop("label"), X_test.pop("label"), X_val.pop("label")
    y_train, y_test, y_val = y_train.values.reshape((-1, 1)), y_test.values.reshape((-1, 1)), y_val.values.reshape((-1, 1))
    
    # Reshape the features for CNN
    X_train = X_train.as_matrix().reshape(X_train.shape[0], 1, 124, 124)
    X_test = X_test.as_matrix().reshape(X_test.shape[0], 1, 124, 124)
    X_val = X_val.as_matrix().reshape(X_val.shape[0], 1, 124, 124)
    
    # Norm data
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_val = X_val.astype('float32')
    X_train /= 255
    X_test /= 255
    X_val /= 255
    
    # Convert labels to categorical values
    y_train = keras.utils.to_categorical(y_train, 2)
    y_test = keras.utils.to_categorical(y_test, 2)
    y_val = keras.utils.to_categorical(y_val, 2)
    
    return X_train, y_train, X_test, y_test, X_val, y_val
    
X_train, y_train, X_test, y_test, X_val, y_val = get_shuffled_splitted_data('data/subset-1-HnxTny.txt.csv')

In [4]:
with tf.device('/gpu:0'):

    import keras
    from keras.preprocessing.image import ImageDataGenerator
    from keras.models import Sequential
    from keras.layers import Dense, Dropout, Activation, Flatten
    from keras.layers import Conv2D, MaxPooling2D
    from keras import backend as K
    from keras.losses import categorical_crossentropy
    from keras.optimizers import Adadelta
    K.set_image_dim_ordering('th')

    batch_size = 32
    num_classes = 2
    epochs = 5

    # The data, shuffled and split between train and test sets:
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')
    print(X_val.shape[0], 'validation samples')

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(1, 124, 124)))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=categorical_crossentropy,
                  optimizer=Adadelta(),
                  metrics=['accuracy'])

    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(X_test, y_test))

    test_score = model.evaluate(X_test, y_test, verbose=0)
    print('Test loss:', test_score[0])
    print('Test accuracy:', test_score[1])
    
    val_score = model.evaluate(X_val, y_val, verbose=0)
    print('Val loss:', val_score[0])
    print('Val accuracy:', val_score[1])

X_train shape: (5318, 1, 124, 124)
5318 train samples
1520 test samples
760 validation samples
Train on 5318 samples, validate on 1520 samples
Epoch 1/5
5318/5318 [==============================] - 66s - loss: 0.0822 - acc: 0.9547 - val_loss: 4.0004e-04 - val_acc: 1.0000
Epoch 2/5
5318/5318 [==============================] - 61s - loss: 0.0452 - acc: 0.9673 - val_loss: 2.3126e-04 - val_acc: 1.0000
Epoch 3/5
5318/5318 [==============================] - 61s - loss: 0.0445 - acc: 0.9677 - val_loss: 3.4764e-05 - val_acc: 1.0000
Epoch 4/5
5318/5318 [==============================] - 61s - loss: 0.0412 - acc: 0.9710 - val_loss: 7.5750e-06 - val_acc: 1.0000
Epoch 5/5
5318/5318 [==============================] - 61s - loss: 0.0400 - acc: 0.9718 - val_loss: 2.3888e-06 - val_acc: 1.0000
Test loss: 2.38881457634e-06
Test accuracy: 1.0
Val loss: 2.8003909859e-06
Val accuracy: 1.0


In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 122, 122)      320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 120, 120)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 60, 60)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 60, 60)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 58, 58)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 128, 29, 29)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 128, 29, 29)       0         
__________